In [1]:
!pip install pdfplumber
!pip install pypdf
!pip install transformers
!pip install torch
!pip install markdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 56.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pypdf
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
from pathlib import Path
import markdown
import pdfplumber
import os
import torch 
from datetime import datetime
from IPython.display import FileLink, display

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
def extract_text_from_pdf(pdf_path):
    """Extract text from PDF with page tracking"""
    text_by_page = {}  # Dictionary to store text for each page
    
    try:
        # Primary method: pdfplumber
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, 1):
                page_text = page.extract_text()
                if page_text and page_text.strip():  # Only store non-empty pages
                    text_by_page[page_num] = page_text
        
        # Backup method: pypdf (if pdfplumber didn't get text)
        if not text_by_page:
            reader = pypdf.PdfReader(pdf_path)
            for page_num, page in enumerate(reader.pages, 1):
                page_text = page.extract_text()
                if page_text and page_text.strip():
                    text_by_page[page_num] = page_text
                
        return text_by_page
    except Exception as e:
        print(f"Error extracting text: {str(e)}")
        return {}

print("PDF extraction function defined!")

PDF extraction function defined!


In [5]:
def contains_mathematical_symbols(text):
    """Check if text contains common mathematical symbols"""
    math_symbols = r'[+\-*/=()<>{}\[\]^√∫∑∏πα-ω]'
    return bool(re.search(math_symbols, text))

print("Mathematical symbol detection function defined!")

Mathematical symbol detection function defined!


In [ ]:
def identify_equations(text_by_page, model, tokenizer):
    """Use LLaMA to identify mathematical equations from text with page numbers"""
    # Set up pad token if not already set
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = tokenizer.pad_token_id
    
    equations_by_page = {}
    batch_size = 4  # Adjust based on your GPU memory
    
    for page_num, text in text_by_page.items():
        page_equations = []
        chunks = [text[i:i+1024] for i in range(0, len(text), 1024)]
        
        # Process chunks in batches
        for i in range(0, len(chunks), batch_size):
            batch_chunks = chunks[i:i + batch_size]
            batch_prompts = [

            f""" you are a mathematical equation identifier. Your task is to identify mathematical equations from the provided text.
                    You will be given a chunk of text, and you need to extract any mathematical equations present in it.
                    you must follow up this instructions:
                    - Do not return any text other than the equations.
                    - If no equations are found, return 'None'.
                    - If you find multiple equations, return each on a new line.
                    - Ensure to include all equations, even if they are complex or involve multiple steps.
                    - Do not include any explanations or additional text.
                    the context is:  
                    Text: {chunk}
                    
                    Equations:""
                            """
                for chunk in batch_chunks
            ]
            
            # Batch process inputs with proper padding
            inputs = tokenizer(
                batch_prompts,
                padding=True,
                truncation=True,
                max_length=1024,
                return_tensors="pt"
            ).to(model.device)
            
            with torch.cuda.amp.autocast():
                outputs = model.generate(
                    **inputs,
                    max_length=1028,
                    num_return_sequences=1,
                    pad_token_id=tokenizer.pad_token_id,
                    do_sample=False
                    
                )
            
            # Process batch outputs
            for output in outputs:
                response = tokenizer.decode(output, skip_special_tokens=True)
                potential_equations = response.split('\n')
                for eq in potential_equations:
                    if eq and eq.strip() != 'None' and contains_mathematical_symbols(eq):
                        page_equations.append(eq.strip())
        
        if page_equations:
            equations_by_page[page_num] = page_equations
    
    return equations_by_page

print("Optimized equation identification function defined with pad token handling!")

Optimized equation identification function defined with pad token handling!


In [12]:
def process_pdf():
    
    # Ask user for PDF path
    pdf_path = "/kaggle/input/same-shit/next-generation-sample-questions-advanced-algebra-and-functions.pdf"
    
    # Check CUDA availability and set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Load the model and tokenizer with optimizations
    model_name = "deepseek-ai/DeepSeek-R1-0528-Qwen3-8B" #  meta-llama/Llama-3.2-3B-Instruct
    
    # Configure model loading with FP16 if CUDA is available
    model_config = {
        "device_map": "auto",
        "use_cache": True,
    }
    
    if torch.cuda.is_available():
        model_config["torch_dtype"] = torch.float16
    
    try:
        # Load tokenizer with proper padding setup
        print("Setting up tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            use_fast=True
        )
        
        # Set up padding token
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            print("Pad token set to EOS token")
        
        # Load model with optimizations
        print("Loading model...")
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            **model_config
        )
        
        # Ensure model knows about pad token
        model.config.pad_token_id = tokenizer.pad_token_id
        
        # Additional optimizations for CUDA
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.backends.cudnn.benchmark = True
        
        print("Model and tokenizer loaded successfully")
        
        # Extract text
        print("Extracting text from PDF...")
        text_by_page = extract_text_from_pdf(pdf_path)
        
        # Extract equations with progress tracking
        print("Identifying equations...")
        total_pages = len(text_by_page)
        equations_by_page = {}
        
        for i, (page_num, text) in enumerate(text_by_page.items(), 1):
            print(f"\rProcessing page {i}/{total_pages}...", end="")
            page_result = identify_equations({page_num: text}, model, tokenizer)
            if page_result:
                equations_by_page.update(page_result)
        print("\nEquation extraction complete!")
        
        # Count total equations 
        total_equations = sum(len(eqs) for eqs in equations_by_page.values())
        
        print(f"\nFound {total_equations} equations across {len(equations_by_page)} pages")
        print("\nEquations by page:")
        print("=" * 50)
        
        for page_num in sorted(equations_by_page.keys()):
            print(f"\nPage {page_num}:")
            print("-" * 20)
            for idx, eq in enumerate(equations_by_page[page_num], 1):
                print(f"Equation {idx}: {eq}")
        
        # Clean up
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        return equations_by_page
        
    except Exception as e:
        print(f"Error during processing: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

print("Optimized main processing function defined!")

Optimized main processing function defined!


In [13]:
# Run the processing and store equations in a global variable
print("Starting PDF processing...")
global_equations = process_pdf()
print("\nProcessing complete!")

Starting PDF processing...
Using device: cuda
Setting up tokenizer...


tokenizer_config.json:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Loading model...


config.json:   0%|          | 0.00/859 [00:00<?, ?B/s]

...

model-00001-of-000002.safetensors:   0%|          | 0.00/8.61G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model and tokenizer loaded successfully
Extracting text from PDF...
Identifying equations...
Processing page 1/10...

/tmp/ipykernel_35/2131887465.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Processing page 10/10...
Equation extraction complete!

Found 531 equations across 10 pages

Equations by page:

Page 1:
--------------------
Equation 1: 1.  2x + 3 = 7
Equation 2: 2.  4y - 5 = 15
Equation 3: 3.  x^2 + 5x + 6 = 0
Equation 4: 4.  y = 2x^2 + 3x - 4
Equation 5: 5.  3x + 2y = 12
Equation 6: 6.  5x^2 - 2x + 1 = 0
Equation 7: 7.  2x + 4y + z = 10
Equation 8: 8.  y = 3x^3 - 2x^2 + x - 1
Equation 9: 9.  2x^2 + 3y^2 = 16
Equation 10: 10.  4x^3 - 3x^2 + 2x - 1 = 0
Equation 11: 11.  x^2 + y^2 = 25
Equation 12: 12.  3x + 4y = 12
Equation 13: 13.  2x^2 + 5x + 3 = 0
Equation 14: 14.  y = x^2 + 2x + 1
Equation 15: 15.  5x + 2y + 3z = 15
Equation 16: 16.  x^3 + 2x^2 - 3x = 0
Equation 17: 17.  2x^2 + 3y^2 + z^2 = 20
Equation 18: 18.  4x^2 - 9y^2 = 0
Equation 19: 19.  3x^2 + 2x - 1 = 0
Equation 20: 20.  y = 2x^2 - 4x + 5
Equation 21: 21.  2x^2 + 3x + 1 = 0
Equation 22: 22.  3x^3 - 2x^2 + x = 0
Equation 23: 23.  x^2 + 4x + 4 = 0
Equation 24: 24.  2x^2 - 5x + 3 = 0
Equation 25: 25.  y = x

In [18]:

def create_and_download_json(equations_by_page):
    if not equations_by_page:
        print("No equations available to save.")
        return
    
    # Create filename with timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'mathematical_equations_{timestamp}.json'
    
    # Prepare JSON data
    json_data = {
        "metadata": {
            "extracted_by": "youssif00",
            "total_pages_with_equations": len(equations_by_page),
            "total_equations": sum(len(eqs) for eqs in equations_by_page.values())
        },
        "equations_by_page": {
            str(page_num): equations
            for page_num, equations in equations_by_page.items()
        }
    }
    
    # Save JSON file
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, indent=4, ensure_ascii=False)
    
    print(f"Created JSON file: {filename}")
    print(f"Total pages with equations: {len(equations_by_page)}")
    print(f"Total equations saved: {json_data['metadata']['total_equations']}")
    
    
    # Create download link
    print("\nClick below to download the JSON file:")
    display(FileLink(filename))
    
    return filename

# Create and download JSON file
if 'global_equations' in globals() and global_equations:
    downloaded_file = create_and_download_json(global_equations)
else:
    print("Please run the PDF processing cell first and make sure it found equations.")
    

Created JSON file: mathematical_equations_20250601_153546.json
Total pages with equations: 10
Total equations saved: 531

Click below to download the JSON file:


/kaggle/working/mathematical_equations_20250601_153546.json